In [1]:
import Pkg

Pkg.add("OSQP")
Pkg.add("Compat")
Pkg.add("BlockDiagonals")

    Updating registry at `C:\Users\Visagio\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Visagio\.julia\environments\v1.7\Project.toml`
  No Changes to `C:\Users\Visagio\.julia\environments\v1.7\Manifest.toml`
   Resolving package versions...
   Installed Compat ─ v3.42.0
    Updating `C:\Users\Visagio\.julia\environments\v1.7\Project.toml`
  [34da2185] + Compat v3.42.0
    Updating `C:\Users\Visagio\.julia\environments\v1.7\Manifest.toml`
  [34da2185] ↑ Compat v3.41.0 ⇒ v3.42.0
Precompiling project...
  ✓ Compat
  ✓ ChainRulesCore
  ✓ ArrayInterface
  ✓ ChangesOfVariables
  ✓ DataStructures
  ✓ FiniteDifferences
  ✓ SortingAlgorithms
  ✓ FiniteDiff
  ✓ QuadGK
  ✓ LogExpFunctions
  ✓ BlockDiagonals
  ✓ StatsBase
  ✓ SpecialFunctions
  ✓ DiffRules
  ✓ StatsFuns
  ✓ ForwardDiff
  ✓ NLSolversBase
  ✓ OptimBase
  ✓ Distributions
  ✓ LsqFit
  ✓ JuMP
  ✓ Plots
  22 dependencies successfully precompiled in 110 seconds (174 already precompiled, 1 sk

In [2]:
using OSQP
using Compat.SparseArrays

In [3]:
# Define problem data

P = sparse([1. -1.; -1. 2.])         # H -- itens de x^2
q = [-2.; -6.]                       # f -- itens de x

# restrições
A = sparse([1. 1.; -1. 2.; 2. 1.])  # A -- multiplica x
l = [0; 0.; 0.]                   # l =< A =< u        ieq
u = [2.; 2.; 3.]                 # l =< A =< u  (b)    ieq

3-element Vector{Float64}:
 2.0
 2.0
 3.0

In [4]:
# Create OSQP object
prob = OSQP.Model()

# Setup workspace and change alpha parameter
OSQP.setup!(prob; P=P, q=q, A=A, l=l,u=u, alpha=1)

# Solve problem
results = OSQP.solve!(prob)

-----------------------------------------------------------------
           OSQP v0.6.2  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 2, constraints m = 3
          nnz(P) + nnz(A) = 9
settings: linear system solver = qdldl,
          eps_abs = 1.0e-003, eps_rel = 1.0e-003,
          eps_prim_inf = 1.0e-004, eps_dual_inf = 1.0e-004,
          rho = 1.00e-001 (adaptive),
          sigma = 1.00e-006, alpha = 1.00, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter  objective    pri res    dua res    rho        time
   1 -9.3904e+000  3.61e-001  3.18e+000  1.00e-001  6.67e-005s
  25 -8.2241e+000  3.08e-003  1.12e-003  1.00e-001  8.42e-005s

status:               solved
number of iterat

OSQP.Results([0.6683944868708552, 1.3326560916454187], [3.1114658407398315, 0.44608743111992627, -9.367506770274764e-18], OSQP.Info(25, :Solved, 1, 0, -8.224117654525562, 0.0030823035800174, 0.0011168048453418233, 4.47e-5, 4.72e-5, 0.0, 0.0, 9.190000000000001e-5, 0, 1.101742596e-315), [NaN, NaN, NaN], [NaN, NaN])

In [5]:
# Define problem data

# H -- itens de x^2
Q_all = 1.0e+06 * sparse([0 0 0 0 0 0; 0 0 0 0 0 0; 0 0 0 0 0 0; 0 0 0 0.0002 0.0018 0.0150; 0 0 0 0.0018 0.0240 0.2250; 0 0 0 0.0150 0.2250 2.2500])      
# f -- itens de x
b_all = [0.; 0.; 0.; 0.; 0.; 0.]                       

# restrições
# Aeq
A_ = 1.0e+03 * sparse([0.0010 0 0 0 0 0; 0 0.0010 0 0 0 0; 0 0 0.0020 0 0 0; 0.0010 0.0050 0.0250 0.1250 0.6250 3.1250; 0 0.0010 0.0100 0.0750 0.5000 3.1250; 0 0 0.0020 0.0300 0.3000 2.5000]) 
# beq
l_ = [0.; 0.; 0.; 2.; 0.; 0.]                   # l =< A =< u        beq
u_ = [0.; 0.; 0.; 2.; 0.; 0.]                   # l =< A =< u  (b)   beq


6-element Vector{Float64}:
 0.0
 0.0
 0.0
 2.0
 0.0
 0.0

In [6]:
# Create OSQP object
q_prob = OSQP.Model()

# Setup workspace and change alpha parameter
OSQP.setup!(q_prob; P=Q_all, q=b_all, A=A_, l=l_, u=u_, alpha=1)

# Solve q_problem
r = OSQP.solve!(q_prob)

-----------------------------------------------------------------
           OSQP v0.6.2  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 6, constraints m = 6
          nnz(P) + nnz(A) = 24
settings: linear system solver = qdldl,
          eps_abs = 1.0e-003, eps_rel = 1.0e-003,
          eps_prim_inf = 1.0e-004, eps_dual_inf = 1.0e-004,
          rho = 1.00e-001 (adaptive),
          sigma = 1.00e-006, alpha = 1.00, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter  objective    pri res    dua res    rho        time
   1  0.0000e+000  2.00e+000  6.61e+004  1.00e-001  5.46e-005s
  25  7.1680e-001  1.29e-014  6.44e-012  1.00e-001  6.72e-005s

status:               solved
number of itera

OSQP.Results([0.0, 7.250496914840555e-20, 8.154508086759954e-20, 0.16000000000000028, -0.048000000000000105, 0.0038400000000000122], [0.7168000000000033, 1.9200000000000086, 1.9200000000000073, -0.7168000000000017, 1.6640000000000088, -1.2800000000000102], OSQP.Info(25, :Solved, 1, 0, 0.7168000000000107, 1.2924229745692364e-14, 6.436900376079254e-12, 4.77e-5, 2.46e-5, 0.0, 0.0, 7.23e-5, 0, 1.184457723e-315), [NaN, NaN, NaN, NaN, NaN, NaN], [NaN, NaN, NaN, NaN, NaN, NaN])

In [7]:
# n_poly
# n_coef
reshape(r.x, (1, 6))

1×6 Matrix{Float64}:
 0.0  7.2505e-20  8.15451e-20  0.16  -0.048  0.00384